In [58]:
import pyabf
import numpy as np
import pandas as pd
import logging




In [12]:
import pyabf
import numpy as np
import pandas as pd
import logging

#####################################################################################
#ABFReader class which should Read an ABF file and return a dictionary containing the data
#####################################################################################

class AbfReader():
    """ Class to load ABF files from path, path string should be inserted to open 
    the abf file with pyabf
    """
    
    def __init__(self, abf_path:str):
        """ abf_path: str -> should be file to open """
        self.abf_path: str = abf_path
        self.initialize_logger()


        try:
            self.abf_path_id: list = self.abf_path.split("/")[-1].split("_")[:2]
            self.abf_path_id: str = "_".join(self.abf_path_id)

        except Exception as e:
            self.abt_path_id = None
            self.logger("ABF File is not correctly formated")
    

        self.abf = None
        self.abf_property_dictionary: dict = None

        
        if self.abf_path:
            try:
                self.load_abf()
            except Exception as e:
                print(f"this is the error: {e}")
        

        if self.abf:
            self.data_table, self.metadata_table = self.get_data_from_sweep()
            self.build_command_epoch_table()
            self.make_membrane_test()

    def initialize_logger(self):
        """ initalized the logger module and write the loger file to abf_file_loaging.log"""
        self.logger = logging.getLogger()
        file_handler = logging.FileHandler('../Logs/abf_file_loaging.log')
        formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        #self.logger.setLevel(logging.error)


    def load_abf(self):
        """ Load the data from the Path and insert into abf object"""
        try:
            self.abf = pyabf.ABF(self.abf_path)
        except:
            self.logger.error("ABF file could not be loaded, Check path or File for potential corruption")
        
        
    def get_data_from_sweep(self):
        """ Extract the data sweeps as table and save it into pd.DataFrame
        """
        data_sweep = pd.DataFrame()
        meta_data_sweep = pd.DataFrame()
        metadata_index = ["dacUnits", 
                          "dataRate",
                          "dataPointCount",
                          "dataLengthSec",
                          "sweepNumber",
                          "sweepLabelY",
                          "sweepLabelX",
                          "sweepUnitsY",
                          "sweepUnitsX",
                          "adcNames",
                          "adcUnits",
                          "CSlow",
                          "YUnit",
                          "XStart",
                          "XInterval",
                          "DataPoints",
                          "Ymin",
                          "Ymax", 
                          "RecordingMode"]
        
        meta_data_sweep["Parameters"] = metadata_index

        columns = 1
        columns_list = []

        for sweep in self.abf.sweepList:
            self.abf.setSweep(sweep)
            sweep_meta_data = self.extract_metadata_parameters(self.abf)
            data = self.abf.sweepY
            time = self.abf.sweepX
            data_sweep[sweep] = data
            meta_data_sweep[sweep] = sweep_meta_data
            columns_list.append("sweep_" + str(columns))
            columns += 1
        data_sweep.index = time
        data_sweep.columns = columns_list
        print(data_sweep)
        data_sweep = data_sweep/1e12
        print(data_sweep)
        meta_data_sweep.columns = ["Parameter"] + columns_list
        meta_data_sweep = meta_data_sweep.set_index("Parameter")
    
        return data_sweep, meta_data_sweep


    def extract_metadata_parameters(self, abf_file):

        """ Extract the metadata from the abf file and return it as a list"""
        metadata_list = []
        
        metadata_list.append(abf_file.dacUnits[0])
        metadata_list.append(abf_file.dataRate)
        metadata_list.append(abf_file.dataPointCount)
        metadata_list.append(abf_file.dataLengthSec)
        metadata_list.append(abf_file.sweepCount)
        metadata_list.append(abf_file.sweepLabelY)
        metadata_list.append(abf_file.sweepLabelX)
        metadata_list.append(abf_file.sweepUnitsY)
        metadata_list.append(abf_file.sweepUnitsX)
        metadata_list.append(abf_file.adcNames[0])
        metadata_list.append(abf_file.adcUnits[0])
        metadata_list.append(1)
        metadata_list.append(abf_file.adcUnits[0][1])
        metadata_list.append(0)
        metadata_list.append(abf_file.dataSecPerPoint) # data rate
        metadata_list.append(abf_file.sweepPointCount) #
        metadata_list.append(np.min(abf_file.sweepY))
        metadata_list.append(np.max(abf_file.sweepY))

        metadata_list.append("3" if abf_file.adcUnits[0] == "pA" else "0")

        return metadata_list
            
    def build_command_epoch_table(self):

        """ retrieve the command epoch table for the analysis
        Epochs Table ramp is indicated as 2, step protocol as 1"""
        epochs_list = [] # epoch lists
        columns_list = ["series_name",
                    "sweep_number",
                    "node_type",
                    "holding_potential",
                    "duration",
                    "increment",
                    "voltage"
                    
        ]
       

        first, last = self.get_first_and_last_sweep()


        
        # append the list with the levels and fill the dataframe
        epochs_list.append([self.abf.protocol for i in range(len(self.abf._epochPerDacSection.fEpochInitLevel))])
        epochs_list.append([self.abf.sweepCount for i in range(len(self.abf._epochPerDacSection.fEpochInitLevel))])
        epochs_list.append(self.abf._epochPerDacSection.nEpochType)
        epochs_list.append([i/1000 for i in self.abf._epochPerDacSection.fEpochInitLevel])
        epochs_list.append([i/10000 for i in self.abf._epochPerDacSection.lEpochInitDuration])
        epochs_list.append([i/1000 for i in self.abf._epochPerDacSection.fEpochLevelInc])
        epochs_list.append([i/1000 for i in self.abf._epochPerDacSection.fEpochInitLevel])
        


        #epochs_list.append(self.abf._epochPerDacSection.nEpochType)
        self.epochs_dataframe = pd.DataFrame(epochs_list, index = columns_list)  
      
        
       
        self.epochs_dataframe.insert(loc = 0,
                                    column="col1",
                                    value = first 
                        )
        self.epochs_dataframe.insert(loc = len(self.epochs_dataframe.columns),
                                    column="col2",
                                    value = last 
                        )
      
        self.epochs_dataframe = self.epochs_dataframe.transpose()
        
            

    def get_first_and_last_sweep(self):
        """ Get the first and last sweep of the abf file"""

        series = self.abf.protocol
        sweep_number = self.abf.sweepCount
        first_potential = self.abf.sweepEpochs.levels[0]
        last_potential = self.abf.sweepEpochs.levels[-1]
        first_time = (self.abf.sweepEpochs.p2s[0] - self.abf.sweepEpochs.p1s[0])/1000
        last_time = (self.abf.sweepEpochs.p2s[-1] - self.abf.sweepEpochs.p1s[-1])/1000
    
        first_command = [series, sweep_number,1,first_potential, first_time, 0, first_potential]
        last_command = [series, sweep_number, 1, last_potential, last_time, 0, last_potential]
        return first_command, last_command




    def make_membrane_test(self):
        """ Dysfunctional for most recording, should record the membrane properties
        But not working for most of the recordings"""
        try:
            memtest = pyabf.tools.memtest.Memtest(self.abf)
            self.abf_property_dictionary["memtest"].append(memtest.CmStep.values)
        except Exception as e:
            self.logger.error("currently not working for the data")

    def get_data_table(self):
        """ Returns the data table as pd.DataFrame"""
        return self.data_table

    def get_metadata_table(self):
        """ Returns the metadata table as pd.DataFrame"""
        return self.metadata_table

    def get_command_epoch_table(self):
        """ Returns the command epoch table as pd.DataFrame"""
        return self.epochs_dataframe

    def get_memtest(self):
        """ Returns the memtest table as list"""
        if self.abf_property_dictionary:
            return self.abf_property_dictionary["memtest"]

    def get_experiment_name(self):
        if self.abf_path_id:
            return self.abf_path_id

    def get_series_name(self):
        return self.abf.protocol
        
        

In [13]:

AbfReader("02FEB2019_0000.abf")


          sweep_1    sweep_2    sweep_3   sweep_4    sweep_5    sweep_6  \
0.0000  -8.544922   1.220703  17.089844  3.051758  -2.441406   3.051758   
0.0001  -3.051758  -4.882812  -9.765625  3.662109   5.493164  -7.934570   
0.0002   3.051758  -8.544922   3.662109 -6.103516 -14.038086   0.000000   
0.0003 -27.465820  13.427734 -10.986328  8.544922  -3.662109  -9.765625   
0.0004  17.700195  -1.220703  20.751953 -9.155273  -6.103516   9.155273   
...           ...        ...        ...       ...        ...        ...   
0.7795 -13.427734  -7.324219   7.324219  7.934570   3.051758  -0.610352   
0.7796  -1.831055  12.817383  -3.051758 -5.493164   3.051758   4.882812   
0.7797  -3.662109 -22.583008  -4.272461  0.000000   0.610352  -3.662109   
0.7798  -2.441406   6.103516   1.220703  8.544922  -6.713867  11.596680   
0.7799 -12.817383  -9.155273   6.103516  1.831055   4.882812  -6.103516   

          sweep_7    sweep_8    sweep_9   sweep_10   sweep_11   sweep_12  \
0.0000   3.051758  -3.6

In [55]:
dictionary = list_of_data_dicts[2]

In [56]:
dictionary.get_command_epoch_table()

,0,1,2
First Level,-60.0,-60.0,80.0
Delta Level,0.0,0.0,0.0
Epoch Duration Increase,0.0,0.0,0.0
Epoch Init Duration,2000.0,1000.0,20000.0
Pulse Period,0.0,0.0,0.0
Pulse Width,0.0,0.0,0.0
Epoch Number,0.0,1.0,2.0
Epoch Type,1.0,1.0,2.0
